In [2]:
import os
print("cwd =", os.getcwd())
print("files =", os.listdir("."))


cwd = /Users/lancy/Desktop/:Users:lancy:Desktop:lancy_slash_project:/lancy-s
files = ['.DS_Store', 'lancy_slash_sample.ipynb', '.gitignore', '.git', 'fannie_mae_sample_300k_raw.csv']


In [3]:
from dotenv import load_dotenv
load_dotenv("/Users/lancy/Desktop/.env", override=True)

import os
print(os.getenv("DB_HOST"))


db.ptukzshzuloxipzwycte.supabase.co


In [4]:
import os
import psycopg2
from dotenv import load_dotenv

# 加载 .env（用绝对路径，最稳）
load_dotenv("/Users/lancy/Desktop/.env", override=True)

# 创建连接
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT", "5432")),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    sslmode="require",
)

print("✅ Database connected")


✅ Database connected


In [5]:
query = """
SELECT *
FROM fannie_mae_cas_raw
TABLESAMPLE SYSTEM (5)
LIMIT 300000;
"""
df = pd.read_sql(query, conn)
print(df.shape)



/var/folders/g1/xtt32sq56_b6b6bts7mwbmxr0000gn/T/ipykernel_43056/523815376.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(300000, 110)


In [6]:
#1 
df.to_csv(
    "fannie_mae_sample_300k_raw.csv",
    index=False
)


In [7]:
df_clean = df.copy()

# 看缺失情况
missing = df_clean.isna().mean().sort_values(ascending=False)
print(missing.head(10))


interest_only_first_principal_and_interest_payment_date    1.0
months_to_amortization                                     1.0
arm_product_type                                           1.0
index                                                      1.0
next_interest_rate_adjustment_date                         1.0
arm_cap_structure                                          1.0
lifetime_interest_rate_cap_up_percent                      1.0
periodic_interest_rate_cap_up_percent                      1.0
arm_plan_number                                            1.0
mortgage_margin                                            1.0
dtype: float64


In [8]:
# 删除 100% 缺失的列
cols_all_missing = missing[missing == 1.0].index
df_clean = df_clean.drop(columns=cols_all_missing)

print(f"Dropped {len(cols_all_missing)} columns with 100% missing values")


Dropped 16 columns with 100% missing values


In [9]:
missing[missing.between(0.05, 0.9)].head(20)


mortgage_insurance_percentage                    0.660247
mortgage_insurance_type                          0.660247
mortgage_insurance_cancellation_indicator        0.660247
co_borrower_credit_score_current                 0.597427
co_borrower_credit_score_at_issuance             0.578163
co_borrower_credit_score_at_origination          0.578000
interest_bearing_upb                             0.381337
payment_deferral_modification_event_indicator    0.339770
borrower_credit_score_current                    0.050733
dtype: float64

In [10]:
from sklearn.impute import SimpleImputer

# ---------- 1️⃣ 明确列分组 ----------

# A. mortgage insurance 相关（缺失 = 无）
mi_cols = [
    "mortgage_insurance_percentage",
    "mortgage_insurance_type",
    "mortgage_insurance_cancellation_indicator",
]

# B. co-borrower credit score（数值）
co_borrower_score_cols = [
    "co_borrower_credit_score_current",
    "co_borrower_credit_score_at_issuance",
    "co_borrower_credit_score_at_origination",
]

# C. 其他数值型（用 median）
median_numeric_cols = [
    "interest_bearing_upb",
    "borrower_credit_score_current",
]

# ---------- 2️⃣ 分别填补 ----------

# A. mortgage insurance A 类：是否存在型变量（缺失本身有意义）0填
df_clean[mi_cols] = df_clean[mi_cols].fillna(
    {
        "mortgage_insurance_percentage": 0,
        "mortgage_insurance_type": "None",
        "mortgage_insurance_cancellation_indicator": 0,
    }
)

# B. co-borrower credit scores → median B 类：信用分数（数值型）median填
median_imputer = SimpleImputer(strategy="median")
df_clean[co_borrower_score_cols] = median_imputer.fit_transform(
    df_clean[co_borrower_score_cols]
)

# C. 其他数值型 → median C 类：金额型（UPB）median填
df_clean[median_numeric_cols] = median_imputer.fit_transform(
    df_clean[median_numeric_cols]
)

print("✅ Partial missing columns imputed")


✅ Partial missing columns imputed


In [11]:
df_clean[mi_cols + co_borrower_score_cols + median_numeric_cols].isna().mean()


mortgage_insurance_percentage                0.0
mortgage_insurance_type                      0.0
mortgage_insurance_cancellation_indicator    0.0
co_borrower_credit_score_current             0.0
co_borrower_credit_score_at_issuance         0.0
co_borrower_credit_score_at_origination      0.0
interest_bearing_upb                         0.0
borrower_credit_score_current                0.0
dtype: float64

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# 数值型 & 类别型列
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df_clean.select_dtypes(include=["object", "category"]).columns

print("Numeric columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))


Numeric columns: 5
Categorical columns: 89


In [13]:
from sklearn.preprocessing import StandardScaler
#数值列填median
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)


In [14]:
cat_cols = df_clean.select_dtypes(include=["object", "category"]).columns
print(f"Categorical columns ({len(cat_cols)}):")
cat_cols

for col in cat_cols:
    print(f"\n=== {col} ===")
    print(df_clean[col].value_counts(dropna=False).head(10))


Categorical columns (89):

=== reference_pool_id ===
reference_pool_id
5361    102042
5350     98965
5353     98779
5363       214
Name: count, dtype: int64

=== loan_identifier ===
loan_identifier
134299044    9
134299041    9
134299040    9
134299039    9
134644693    8
134644694    8
134640254    8
134640256    8
134640257    8
134640260    8
Name: count, dtype: int64

=== monthly_reporting_period ===
monthly_reporting_period
32023     13654
22025     12781
32024     12651
52024     12566
22024     12446
32025     12420
62023     12315
112024    12192
112023    12128
12024     12028
Name: count, dtype: int64

=== channel ===
channel
R    201750
C     58680
B     39570
Name: count, dtype: int64

=== seller_name ===
seller_name
Other                                        133513
Rocket Mortgage, LLC                          30629
United Wholesale Mortgage, LLC                16584
Wells Fargo Bank, N.A.                        11320
JPMorgan Chase Bank, National Association      8332
l

In [15]:
df_clean.select_dtypes(include=["object","category"]).nunique().sort_values(ascending=False).head(20)


current_actual_upb                      269637
loan_identifier                         172253
upb_at_issuance                         146061
total_principal_current                 107709
scheduled_principal_current             101165
unscheduled_principal_current            21196
upb_at_the_time_of_removal                9585
loan_payment_history                      4263
original_interest_rate                    1592
current_interest_rate                     1578
original_upb                              1001
zip_code_short                             882
total_deferral_amount                      576
remaining_months_to_maturity               453
metropolitan_statistical_area_msa          393
borrower_credit_score_at_origination       232
remaining_months_to_legal_maturity         193
maturity_date                              158
cumulative_modification_loss_amount         99
original_loan_term                          92
dtype: int64

In [16]:
#明确“真正的 categorical 能onehot的
cat_cols = (
    df_clean
    .select_dtypes(include=["object","category"])
    .columns
)

# 用 unique 数过滤
cat_cardinality = df_clean[cat_cols].nunique()

# 只保留低基数 categorical
LOW_CARD_MAX = 50
true_cat_cols = cat_cardinality[cat_cardinality <= LOW_CARD_MAX].index.tolist()

print("True categorical cols (for One-Hot):")
print(true_cat_cols)


True categorical cols (for One-Hot):
['reference_pool_id', 'monthly_reporting_period', 'channel', 'seller_name', 'servicer_name', 'master_servicer', 'origination_date', 'first_payment_date', 'original_loan_to_value_ratio_ltv', 'original_combined_loan_to_value_ratio_cltv', 'number_of_borrowers', 'first_time_home_buyer_indicator', 'loan_purpose', 'property_type', 'number_of_units', 'occupancy_status', 'mortgage_insurance_percentage', 'amortization_type', 'prepayment_penalty_indicator', 'interest_only_loan_indicator', 'current_loan_delinquency_status', 'modification_flag', 'mortgage_insurance_cancellation_indicator', 'zero_balance_code', 'zero_balance_effective_date', 'last_paid_installment_date', 'foreclosure_date', 'disposition_date', 'foreclosure_costs', 'property_preservation_and_repair_costs', 'asset_recovery_costs', 'miscellaneous_holding_expenses_and_credits', 'associated_taxes_for_holding_property', 'net_sales_proceeds', 'credit_enhancement_proceeds', 'repurchase_make_whole_procee

In [17]:
#把“假 categorical”转回数值 or 删除
numeric_like_cols = [
    "current_actual_upb",
    "upb_at_issuance",
    "total_principal_current",
    "scheduled_principal_current",
    "unscheduled_principal_current",
    "original_interest_rate",
    "current_interest_rate",
    "original_upb",
]

for c in numeric_like_cols:
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")


In [18]:
#id类全drop
id_like_cols = [
    "loan_identifier",
    "loan_payment_history",
]

df_clean = df_clean.drop(columns=id_like_cols)


In [19]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer

TARGET = "zero_balance_code"

# 1) 先构造 y 和 X（非常关键：列清单必须基于最终 X）
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 建议同时移除可能泄漏的日期列（可选但推荐）
leak_cols = ["zero_balance_effective_date", "zero_balance_code_change_date"]
drop_cols = [TARGET] + [c for c in leak_cols if c in df_model.columns]

X = df_model.drop(columns=drop_cols)

# 2) 在 X 上重新算列类型（不要用 df_clean）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("num:", len(num_cols), "cat:", len(cat_cols))

# 3) 预处理（RF 不需要 scaler，所以数值只 impute 就够）
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str), validate=False)),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)


num: 13 cat: 76


In [20]:
import pandas as pd
import numpy as np

TARGET = "zero_balance_code"

# 1) 先构造标签 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")

# 这里的规则你也可以改：非空且 != 0 视为 “发生 zero balance（还清/终止）”
y = ((~z.isna()) & (z != 0)).astype(int)
print("Label distribution:\n", y.value_counts())

# 2) 再构造特征 X（一定要把 target 列从特征里去掉）
X = df_model.drop(columns=[TARGET])

# 3) 关键：基于 X 重新计算列类型（不要用旧的 num_cols/cat_cols）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("num cols:", len(num_cols), "cat cols:", len(cat_cols))
print("Does TARGET appear in cols lists?",
      TARGET in num_cols, TARGET in cat_cols)  # 应该都是 False


Label distribution:
 zero_balance_code
0    286117
1     13883
Name: count, dtype: int64
num cols: 13 cat cols: 78
Does TARGET appear in cols lists? False False


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

TARGET = "zero_balance_code"

# 1) 构造 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 2) 构造 X（去掉 target + 建议去掉泄漏日期列）
leak_cols = ["zero_balance_effective_date", "zero_balance_code_change_date"]
drop_cols = [TARGET] + [c for c in leak_cols if c in df_model.columns]
X = df_model.drop(columns=drop_cols)

# 3) 列类型（必须基于 X）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

# ✅ 关键硬修复：把所有 cat 列统一成字符串（彻底杜绝 int/str 混合）
X[cat_cols] = X[cat_cols].astype("string")
# 用一个明确的占位符处理缺失（这样 onehot 会把它当成一个类别）
X[cat_cols] = X[cat_cols].fillna("MISSING")

print("num:", len(num_cols), "cat:", len(cat_cols))

# 4) 预处理
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline([
    # 这里 imputer 可有可无（你已经 fillna 了），保留也没问题
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# 5) 切分
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6) Random Forest
rf = RandomForestClassifier(
    n_estimators=300,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced",
    min_samples_leaf=50,
)

model = Pipeline([
    ("preprocess", preprocessor),
    ("rf", rf),
])

model.fit(X_train, y_train)
print("✅ Random Forest trained")


num: 13 cat: 76
✅ Random Forest trained


In [22]:
#最后一步
from sklearn.metrics import roc_auc_score, classification_report

proba = model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, proba))
print(classification_report(y_test, model.predict(X_test)))


ROC-AUC: 0.9999999999999999
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57223
           1       1.00      1.00      1.00      2777

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000



In [23]:
# No leaking
import re
import pandas as pd
import numpy as np

TARGET = "zero_balance_code"

df_model = df_clean.copy()

# ---- 1) 标签：是否发生 zero balance（非空且不为0）----
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# ---- 2) 明确要删的“泄漏列”（强烈建议删）----
explicit_drop = [
    TARGET,
    "zero_balance_effective_date",
    "zero_balance_code_change_date",
    # 如果你有其它 zero_balance 相关列，也一起删
]

# ---- 3) 规则删：删掉典型“后验/未来信息”列（按列名模式）----
leak_patterns = [
    r"payment_history",             # 还款历史字符串
    r"_current$",                   # *_current 往往是当前时点信息
    r"current_",                    # current_*
    r"at_the_time_of_removal",      # removal 时点才有
    r"removal",                     # removal相关
    r"modification",                # 修改后才有
    r"deferral",                    # 展期/延期事件后
    r"cumulative",                  # 累计值通常是事后
    r"non_interest_bearing",        # 通常事后状态
    r"interest_bearing_upb",        # 很多场景是后续月度表现
    r"total_principal_current",     # 你之前也见过 *_current
    r"scheduled_principal_current",
    r"unscheduled_principal_current",
    r"upb_at_the_time_of_removal",
]

pattern = re.compile("|".join(leak_patterns), flags=re.IGNORECASE)

pattern_drop = [c for c in df_model.columns if pattern.search(c)]

# 合并 drop 列，并只保留 df 中真实存在的
drop_cols = [c for c in set(explicit_drop + pattern_drop) if c in df_model.columns]

print("Dropping columns (count):", len(drop_cols))
print("Example dropped:", drop_cols[:20])

# ---- 4) 构造 X（去泄漏后的特征）----
X = df_model.drop(columns=drop_cols)

print("X shape:", X.shape)
print("Label distribution:\n", y.value_counts())


Dropping columns (count): 23
Example dropped: ['modification_related_non_interest_bearing_upb', 'co_borrower_credit_score_current', 'current_loan_delinquency_status', 'current_period_modification_loss_amount', 'current_actual_upb', 'unscheduled_principal_current', 'zero_balance_code', 'cumulative_credit_event_net_gain_or_loss', 'current_interest_rate', 'current_list_price', 'modification_flag', 'total_deferral_amount', 'borrower_credit_score_current', 'upb_at_the_time_of_removal', 'zero_balance_effective_date', 'scheduled_principal_current', 'current_list_start_date', 'current_period_credit_event_net_gain_or_loss', 'total_principal_current', 'zero_balance_code_change_date']
X shape: (300000, 69)
Label distribution:
 zero_balance_code
0    286117
1     13883
Name: count, dtype: int64


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

# 切分
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 列类型（基于 X！）
num_cols = X_train.select_dtypes(include=["int64","float64","bool"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object","category"]).columns.tolist()

# 硬统一：类别列转 string + 缺失占位符（防止 int/str 混合再次出现）
X_train = X_train.copy()
X_test = X_test.copy()
X_train[cat_cols] = X_train[cat_cols].astype("string").fillna("MISSING")
X_test[cat_cols]  = X_test[cat_cols].astype("string").fillna("MISSING")

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

rf = RandomForestClassifier(
    n_estimators=400,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced",
    min_samples_leaf=50
)

model_noleak = Pipeline([
    ("preprocess", preprocess),
    ("rf", rf),
])

model_noleak.fit(X_train, y_train)

proba = model_noleak.predict_proba(X_test)[:, 1]
pred = model_noleak.predict(X_test)

print("✅ No-leak RF ROC-AUC:", roc_auc_score(y_test, proba))
print(classification_report(y_test, pred))


✅ No-leak RF ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57223
           1       1.00      1.00      1.00      2777

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000



In [25]:
# ============================================
# 方案二：只使用贷款发放时的特征（无泄漏版本）
# ============================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

TARGET = "zero_balance_code"

# 1) 构造标签 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 2) 只保留贷款发放时就已知的特征（origination-time features）
origination_features = [
    # 贷款金额相关
    "original_upb",
    
    # 利率相关
    "original_interest_rate",
    
    # LTV 相关
    "original_loan_to_value_ratio_ltv",
    "original_combined_loan_to_value_ratio_cltv",
    
    # 信用分数（发放时）
    "borrower_credit_score_at_origination",
    "co_borrower_credit_score_at_origination",
    "borrower_credit_score_at_issuance",
    "co_borrower_credit_score_at_issuance",
    
    # 借款人信息
    "number_of_borrowers",
    "first_time_home_buyer_indicator",
    
    # 贷款信息
    "loan_purpose",
    "channel",
    "amortization_type",
    "prepayment_penalty_indicator",
    "interest_only_loan_indicator",
    
    # 房产信息
    "property_type",
    "number_of_units",
    "occupancy_status",
    "zip_code_short",
    "metropolitan_statistical_area_msa",
    
    # 保险相关
    "mortgage_insurance_percentage",
    "mortgage_insurance_type",
    
    # 其他发放时信息
    "seller_name",
    "servicer_name",
    "origination_date",
    "first_payment_date",
    "maturity_date",
    "remaining_months_to_maturity",
    "high_balance_loan_indicator",
    "homeready_program_indicator",
    "relocation_mortgage_indicator",
]

# 只保留数据集中存在的列
available_features = [c for c in origination_features if c in df_model.columns]
print(f"使用的特征数量: {len(available_features)}")
print(f"特征列表: {available_features}")

# 3) 构造 X
X = df_model[available_features].copy()

# 4) 列类型
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print(f"\n数值列 ({len(num_cols)}): {num_cols}")
print(f"类别列 ({len(cat_cols)}): {cat_cols}")

# 5) 类别列统一处理
X[cat_cols] = X[cat_cols].astype("string").fillna("MISSING")

# 6) 划分数据
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n训练集: {X_train.shape}, 测试集: {X_test.shape}")
print(f"标签分布 - 训练集: {y_train.value_counts().to_dict()}")
print(f"标签分布 - 测试集: {y_test.value_counts().to_dict()}")

使用的特征数量: 31
特征列表: ['original_upb', 'original_interest_rate', 'original_loan_to_value_ratio_ltv', 'original_combined_loan_to_value_ratio_cltv', 'borrower_credit_score_at_origination', 'co_borrower_credit_score_at_origination', 'borrower_credit_score_at_issuance', 'co_borrower_credit_score_at_issuance', 'number_of_borrowers', 'first_time_home_buyer_indicator', 'loan_purpose', 'channel', 'amortization_type', 'prepayment_penalty_indicator', 'interest_only_loan_indicator', 'property_type', 'number_of_units', 'occupancy_status', 'zip_code_short', 'metropolitan_statistical_area_msa', 'mortgage_insurance_percentage', 'mortgage_insurance_type', 'seller_name', 'servicer_name', 'origination_date', 'first_payment_date', 'maturity_date', 'remaining_months_to_maturity', 'high_balance_loan_indicator', 'homeready_program_indicator', 'relocation_mortgage_indicator']

数值列 (4): ['original_upb', 'original_interest_rate', 'co_borrower_credit_score_at_origination', 'co_borrower_credit_score_at_issuance']
类别

In [26]:
# 7) 构建预处理管道
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),  # RF 其实不需要，但加上也无妨
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# 8) 随机森林模型
rf = RandomForestClassifier(
    n_estimators=400,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced",  # 处理类别不平衡
    min_samples_leaf=50,
    max_depth=15,  # 限制深度防止过拟合
)

model_origination = Pipeline([
    ("preprocess", preprocessor),
    ("rf", rf),
])

# 9) 训练
print("开始训练模型（只用发放时特征）...")
model_origination.fit(X_train, y_train)
print("✅ 训练完成！")

开始训练模型（只用发放时特征）...
✅ 训练完成！


In [27]:
# 10) 评估模型
proba = model_origination.predict_proba(X_test)[:, 1]
pred = model_origination.predict(X_test)

roc_auc = roc_auc_score(y_test, proba)

print("=" * 60)
print("📊 只用贷款发放时特征的模型结果")
print("=" * 60)
print(f"\n🎯 ROC-AUC: {roc_auc:.4f}")
print("\n分类报告:")
print(classification_report(y_test, pred))

# 对比之前的结果
print("=" * 60)
print("📈 结果对比")
print("=" * 60)
print(f"之前（有泄漏）ROC-AUC:     0.9999997431")
print(f"现在（无泄漏）ROC-AUC:     {roc_auc:.4f}")
print()

if roc_auc < 0.95:
    print("✅ 现在的 AUC 更合理了！之前的完美分数确实是数据泄漏导致的。")
else:
    print("⚠️ AUC 仍然很高，可能还有其他泄漏源，需要进一步检查。")

📊 只用贷款发放时特征的模型结果

🎯 ROC-AUC: 1.0000

分类报告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57223
           1       1.00      1.00      1.00      2777

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000

📈 结果对比
之前（有泄漏）ROC-AUC:     0.9999997431
现在（无泄漏）ROC-AUC:     1.0000

⚠️ AUC 仍然很高，可能还有其他泄漏源，需要进一步检查。


In [36]:
# 11) 查看特征重要性（无需 matplotlib）

# 获取特征名
feature_names = model_origination.named_steps['preprocess'].get_feature_names_out()
importances = model_origination.named_steps['rf'].feature_importances_

# 创建 DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

# 显示 Top 20 重要特征
print("=" * 60)
print("🔍 Top 20 最重要特征")
print("=" * 60)
print(importance_df.head(20).to_string(index=False))

🔍 Top 20 最重要特征
                                          feature  importance
                       cat__maturity_date_MISSING    0.217509
                       cat__servicer_name_MISSING    0.184858
        cat__remaining_months_to_maturity_MISSING    0.181738
                         cat__servicer_name_Other    0.052681
                       cat__maturity_date_12052.0    0.049479
                       cat__maturity_date_22052.0    0.049141
                       cat__maturity_date_32052.0    0.025283
                       cat__maturity_date_42052.0    0.017127
          cat__servicer_name_Rocket Mortgage, LLC    0.011347
       cat__servicer_name_JPMorgan Chase Bank, NA    0.011097
  cat__servicer_name_Lakeview Loan Servicing, LLC    0.009182
                       cat__maturity_date_52052.0    0.008076
                      cat__maturity_date_122051.0    0.007834
      cat__servicer_name_Onslow Bay Financial LLC    0.007216
       cat__servicer_name_Nationstar Mortgage LLC    0.

In [37]:
# 12) 混淆矩阵（纯文字版本，无需 matplotlib）
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred)

print("=" * 60)
print("📊 混淆矩阵 (Confusion Matrix)")
print("=" * 60)
print()
print("                 预测值")
print("              正常(0)  终止(1)")
print(f"实际 正常(0)   {cm[0,0]:>6}   {cm[0,1]:>6}")
print(f"     终止(1)   {cm[1,0]:>6}   {cm[1,1]:>6}")
print()

# 计算各项指标
tn, fp, fn, tp = cm.ravel()
print("=" * 60)
print("📈 详细指标")
print("=" * 60)
print(f"True Negatives (TN):  {tn:>6}  - 正确预测为正常")
print(f"False Positives (FP): {fp:>6}  - 误报为终止")
print(f"False Negatives (FN): {fn:>6}  - 漏报（实际终止但预测正常）")
print(f"True Positives (TP):  {tp:>6}  - 正确预测为终止")
print()

sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"Sensitivity (Recall): {sensitivity:.4f}  - 终止贷款的识别率")
print(f"Specificity:          {specificity:.4f}  - 正常贷款的识别率")

print()
print("=" * 60)
print("📝 说明")
print("=" * 60)
print("- 如果 ROC-AUC 在 0.65-0.80 之间，这是房贷违约预测的正常水平")
print("- 如果 ROC-AUC > 0.95，可能还有隐藏的数据泄漏")

📊 混淆矩阵 (Confusion Matrix)

                 预测值
              正常(0)  终止(1)
实际 正常(0)    57223        0
     终止(1)        0     2777

📈 详细指标
True Negatives (TN):   57223  - 正确预测为正常
False Positives (FP):      0  - 误报为终止
False Negatives (FN):      0  - 漏报（实际终止但预测正常）
True Positives (TP):    2777  - 正确预测为终止

Sensitivity (Recall): 1.0000  - 终止贷款的识别率
Specificity:          1.0000  - 正常贷款的识别率

📝 说明
- 如果 ROC-AUC 在 0.65-0.80 之间，这是房贷违约预测的正常水平
- 如果 ROC-AUC > 0.95，可能还有隐藏的数据泄漏


In [38]:
# ============================================
# 🔒 更严格版本：只用绝对安全的发放时特征
# ============================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

TARGET = "zero_balance_code"

# 1) 构造标签 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 2) 🔒 只保留绝对安全的发放时静态特征（排除所有日期和可能变化的字段）
safe_origination_features = [
    # 贷款金额（发放时固定）
    "original_upb",
    
    # 利率（发放时固定）
    "original_interest_rate",
    
    # LTV（发放时固定）
    "original_loan_to_value_ratio_ltv",
    "original_combined_loan_to_value_ratio_cltv",
    
    # 信用分数（发放时）
    "borrower_credit_score_at_origination",
    "co_borrower_credit_score_at_origination",
    
    # 借款人信息（发放时固定）
    "number_of_borrowers",
    "first_time_home_buyer_indicator",
    
    # 贷款类型信息（发放时固定）
    "loan_purpose",
    "channel",
    "amortization_type",
    
    # 房产信息（发放时固定）
    "property_type",
    "number_of_units",
    "occupancy_status",
    
    # 保险（发放时）
    "mortgage_insurance_percentage",
]

# 只保留数据集中存在的列
available_features = [c for c in safe_origination_features if c in df_model.columns]
print(f"✅ 使用的安全特征数量: {len(available_features)}")
print(f"特征列表: {available_features}")

# 3) 关键：每笔贷款只保留第一条记录（发放时的快照）
# 如果有 loan_identifier，按它去重；否则用所有特征去重
df_model_dedup = df_model.drop_duplicates(subset=available_features, keep='first')
print(f"\n原始数据行数: {len(df_model)}")
print(f"去重后行数: {len(df_model_dedup)}")

# 重新构造 X 和 y（基于去重后的数据）
X = df_model_dedup[available_features].copy()
y = y.loc[df_model_dedup.index]

print(f"\n标签分布:")
print(y.value_counts())

✅ 使用的安全特征数量: 15
特征列表: ['original_upb', 'original_interest_rate', 'original_loan_to_value_ratio_ltv', 'original_combined_loan_to_value_ratio_cltv', 'borrower_credit_score_at_origination', 'co_borrower_credit_score_at_origination', 'number_of_borrowers', 'first_time_home_buyer_indicator', 'loan_purpose', 'channel', 'amortization_type', 'property_type', 'number_of_units', 'occupancy_status', 'mortgage_insurance_percentage']

原始数据行数: 300000
去重后行数: 172132

标签分布:
zero_balance_code
0    164633
1      7499
Name: count, dtype: int64


In [39]:
# 4) 列类型
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print(f"数值列 ({len(num_cols)}): {num_cols}")
print(f"类别列 ({len(cat_cols)}): {cat_cols}")

# 5) 类别列统一处理
X[cat_cols] = X[cat_cols].astype("string").fillna("MISSING")

# 6) 划分数据
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n训练集: {X_train.shape}, 测试集: {X_test.shape}")

# 7) 构建预处理管道
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# 8) 随机森林模型
rf = RandomForestClassifier(
    n_estimators=300,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced",
    min_samples_leaf=50,
    max_depth=10,
)

model_safe = Pipeline([
    ("preprocess", preprocessor),
    ("rf", rf),
])

# 9) 训练
print("\n开始训练模型（严格安全特征版本）...")
model_safe.fit(X_train, y_train)
print("✅ 训练完成！")

数值列 (3): ['original_upb', 'original_interest_rate', 'co_borrower_credit_score_at_origination']
类别列 (12): ['original_loan_to_value_ratio_ltv', 'original_combined_loan_to_value_ratio_cltv', 'borrower_credit_score_at_origination', 'number_of_borrowers', 'first_time_home_buyer_indicator', 'loan_purpose', 'channel', 'amortization_type', 'property_type', 'number_of_units', 'occupancy_status', 'mortgage_insurance_percentage']

训练集: (137705, 15), 测试集: (34427, 15)

开始训练模型（严格安全特征版本）...
✅ 训练完成！


In [40]:
# 10) 评估模型
proba = model_safe.predict_proba(X_test)[:, 1]
pred = model_safe.predict(X_test)

roc_auc = roc_auc_score(y_test, proba)

print("=" * 60)
print("📊 严格安全特征版本的模型结果")
print("=" * 60)
print(f"\n🎯 ROC-AUC: {roc_auc:.4f}")
print("\n分类报告:")
print(classification_report(y_test, pred))

# 混淆矩阵
cm = confusion_matrix(y_test, pred)
print("=" * 60)
print("📊 混淆矩阵")
print("=" * 60)
print(f"\n              预测值")
print(f"           正常(0)  终止(1)")
print(f"实际 正常(0) {cm[0,0]:>6}   {cm[0,1]:>6}")
print(f"     终止(1) {cm[1,0]:>6}   {cm[1,1]:>6}")

# 结果解读
print("\n" + "=" * 60)
print("📝 结果解读")
print("=" * 60)
if roc_auc > 0.99:
    print("⚠️  ROC-AUC 仍然接近 1.0！")
    print("    可能原因：")
    print("    1. 数据中同一笔贷款的多条记录导致信息泄漏")
    print("    2. 目标变量的定义本身有问题")
    print("    3. 某些特征在贷款终止后被回填")
elif roc_auc > 0.95:
    print("⚠️  ROC-AUC > 0.95，仍然偏高，可能存在轻微泄漏")
elif roc_auc > 0.80:
    print("✅ ROC-AUC 在 0.80-0.95 之间，比较合理但偏高")
else:
    print("✅ ROC-AUC 在正常范围内！这是真实的预测能力。")

📊 严格安全特征版本的模型结果

🎯 ROC-AUC: 0.6074

分类报告:
              precision    recall  f1-score   support

           0       0.97      0.51      0.66     32927
           1       0.06      0.64      0.10      1500

    accuracy                           0.51     34427
   macro avg       0.51      0.58      0.38     34427
weighted avg       0.93      0.51      0.64     34427

📊 混淆矩阵

              预测值
           正常(0)  终止(1)
实际 正常(0)  16655    16272
     终止(1)    533      967

📝 结果解读
✅ ROC-AUC 在正常范围内！这是真实的预测能力。


In [41]:
# 11) 特征重要性分析
feature_names = model_safe.named_steps['preprocess'].get_feature_names_out()
importances = model_safe.named_steps['rf'].feature_importances_

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("=" * 60)
print("🔍 Top 15 最重要特征")
print("=" * 60)
print(importance_df.head(15).to_string(index=False))

print("\n" + "=" * 60)
print("💡 如果 ROC-AUC 仍然接近 1.0，检查以下几点：")
print("=" * 60)
print("1. 查看你的数据：每笔贷款有多少条记录？")
print("   运行: df_clean.groupby('reference_pool_id').size().describe()")
print()
print("2. 检查目标变量：zero_balance_code 的分布是否合理？")
print("   运行: df_clean['zero_balance_code'].value_counts()")
print()
print("3. 这个预测任务可能本身就很简单：")
print("   - 如果数据只包含已经终止的贷款记录，模型可能在'预测'已知结果")

🔍 Top 15 最重要特征
                                           feature  importance
                                 num__original_upb    0.135879
            cat__first_time_home_buyer_indicator_N    0.132278
            cat__first_time_home_buyer_indicator_Y    0.130265
                       num__original_interest_rate    0.088316
      num__co_borrower_credit_score_at_origination    0.032336
                               cat__loan_purpose_P    0.027726
                               cat__loan_purpose_C    0.026402
                           cat__occupancy_status_P    0.023106
              cat__mortgage_insurance_percentage_0    0.019326
                                    cat__channel_B    0.017422
                             cat__property_type_SF    0.017191
                                    cat__channel_R    0.013898
cat__original_combined_loan_to_value_ratio_cltv_97    0.013061
          cat__original_loan_to_value_ratio_ltv_97    0.010974
                             cat__proper